In [5]:
import sys
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install pandas numpy scipy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------------- ------------------------- 3.4/9.7 MB 25.1 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 42.7 MB/s  0:00:00
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   ------------------- -------------------- 17.3/36.3 MB 83.2 MB/s eta 0:00:01
   ---------------------------------- ----- 30.9/36.3 MB 74.3 MB/s eta 0:00:01
   ---------------------------------------- 36.3/36.3 MB 77.0 MB/s  0:00:00

   ---------------------------------------- 0/2 [scipy]
   ---------------------------------------- 0/2 [scipy]
   ---------------------------------------- 0/2 [scipy]
   ---------------------------------------- 0/2 [scipy]
   ---------------------------------------- 0/2 [scipy]
   ------------------------------------

In [7]:
import os
import pandas as pd
import numpy as np
from scipy.fft import rfft, rfftfreq

In [11]:
# Base URL is environment-controlled
BASE_URL = os.getenv(
    "DATA_BASE_URL",
    "https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data"  # safe default for GitHub/notebooks
)

files = [
    f"{BASE_URL}/harmonic_oscillator_simulation.csv",
    f"{BASE_URL}/harmonic_oscillator_simulation_2.csv",
    f"{BASE_URL}/harmonic_oscillator_simulation_3.csv",
    f"{BASE_URL}/harmonic_oscillator_simulation_4.csv",
    f"{BASE_URL}/harmonic_oscillator_simulation_5.csv",
]

print("✓ Base URL set to:")
print(BASE_URL)
print("\n✓ Files to be loaded:")

for f in files:
    print("  -", f)

print("\nTesting data access...\n")

for i, file_url in enumerate(files, start=1):
    try:
        df_test = pd.read_csv(file_url)
        print(
            f"✓ File {i} loaded successfully "
            f"({df_test.shape[0]} rows, {df_test.shape[1]} columns)"
        )
    except Exception as e:
        print(f"✗ File {i} failed to load:")
        print(f"  {file_url}")
        print(f"  Error: {e}")
        

✓ Base URL set to:
https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data

✓ Files to be loaded:
  - https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data/harmonic_oscillator_simulation.csv
  - https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data/harmonic_oscillator_simulation_2.csv
  - https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data/harmonic_oscillator_simulation_3.csv
  - https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data/harmonic_oscillator_simulation_4.csv
  - https://raw.githubusercontent.com/melab-cmu/vibration-signal/main/data/harmonic_oscillator_simulation_5.csv

Testing data access...

✓ File 1 loaded successfully (5000 rows, 2 columns)
✓ File 2 loaded successfully (100 rows, 2 columns)
✓ File 3 loaded successfully (500 rows, 2 columns)
✓ File 4 loaded successfully (100 rows, 2 columns)
✓ File 5 loaded successfully (2500 rows, 2 columns)


In [12]:
results = []
for file_url in files:
    df = pd.read_csv(file_url)

     # Keep only first two columns; coerce to numeric in case of string headers/formatting
    df = df.iloc[:, :2].copy()
    df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors="coerce")
    df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors="coerce")
    df = df.dropna()

    time = df.iloc[:, 0].values
    signal = df.iloc[:, 1].values

    # Sampling rate
    dt = np.mean(np.diff(time))
    fs = 1.0 / dt

    # Frequency (FFT)
    yf = np.abs(rfft(signal - np.mean(signal)))
    xf = rfftfreq(len(signal), dt)
    dominant_freq = xf[np.argmax(yf)]

    # Amplitude
    amplitude = (np.max(signal) - np.min(signal)) / 2

    # ADC input range
    adc_min = np.min(signal)
    adc_max = np.max(signal)

    results.append({
        "source": file_url,
        "sampling_rate_hz": fs,
        "dominant_frequency_hz": dominant_freq,
        "amplitude": amplitude,
        "adc_min": adc_min,
        "adc_max": adc_max
    })

In [13]:
results_df = pd.DataFrame(results)
results_df

,source,sampling_rate_hz,dominant_frequency_hz,amplitude,adc_min,adc_max
0,https://raw.githubusercontent.com/melab-cmu/vi...,999.8,1.9996,0.006760,-0.006760,0.006760
1,https://raw.githubusercontent.com/melab-cmu/vi...,19.8,4.9500,0.005117,-0.005117,0.005117
2,https://raw.githubusercontent.com/melab-cmu/vi...,99.8,0.9980,0.067830,-0.067831,0.067830
3,https://raw.githubusercontent.com/melab-cmu/vi...,19.8,8.9100,0.003105,-0.003104,0.003106
4,https://raw.githubusercontent.com/melab-cmu/vi...,499.8,199.9200,0.000001,-0.000001,0.000001
